# Librairie

In [ ]:
import streamlit as st
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Charger le modèle sauvegardé
model = joblib.load('RF_model.joblib')
RF_model.joblib

# Définir les colonnes numériques et catégorielles
numeric_features = ['MONTH', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'DISTANCE']
categorical_features = ['YEAR', 'QUARTER', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'UNIQUE_CARRIER', 
                        'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 
                        'DEST_AIRPORT_ID', 'DEST_CITY_NAME', 'DEST_STATE_ABR']

# Pipeline pour les caractéristiques numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline pour les caractéristiques catégorielles
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinaison des transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Fonction pour prédire le retard
def predict_delay(input_data):
    # Transformation des données d'entrée
    input_transformed = preprocessor.transform(input_data)
    # Prédiction
    prediction = model.predict(input_transformed)
    return prediction[0]

# Interface utilisateur Streamlit
st.title('Prédiction de Retard de Vol')

st.header('Veuillez entrer les informations de votre vol :')

# Collecte des données utilisateur
year = st.selectbox('Année', ['2020', '2021', '2022', '2023'])
quarter = st.selectbox('Trimestre', ['1', '2', '3', '4'])
month = st.slider('Mois', 1, 12, 1)
day_of_month = st.slider('Jour du Mois', 1, 31, 1)
day_of_week = st.selectbox('Jour de la Semaine', ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'])
unique_carrier = st.text_input('Code Compagnie Aérienne')
origin_airport_id = st.text_input('ID Aéroport d\'Origine')
origin_city_name = st.text_input('Nom de la Ville d\'Origine')
origin_state_abr = st.text_input('Code État d\'Origine')
dest_airport_id = st.text_input('ID Aéroport de Destination')
dest_city_name = st.text_input('Nom de la Ville de Destination')
dest_state_abr = st.text_input('Code État de Destination')
crs_dep_time = st.number_input('Heure de Départ Planifiée (HHMM)', min_value=0, max_value=2359, step=1)
crs_arr_time = st.number_input('Heure d\'Arrivée Planifiée (HHMM)', min_value=0, max_value=2359, step=1)
distance = st.number_input('Distance (miles)', min_value=0.0)

# Création d'un DataFrame pour les données d'entrée
input_data = pd.DataFrame({
    'YEAR': [year],
    'QUARTER': [quarter],
    'MONTH': [month],
    'DAY_OF_MONTH': [day_of_month],
    'DAY_OF_WEEK': [day_of_week],
    'UNIQUE_CARRIER': [unique_carrier],
    'ORIGIN_AIRPORT_ID': [origin_airport_id],
    'ORIGIN_CITY_NAME': [origin_city_name],
    'ORIGIN_STATE_ABR': [origin_state_abr],
    'DEST_AIRPORT_ID': [dest_airport_id],
    'DEST_CITY_NAME': [dest_city_name],
    'DEST_STATE_ABR': [dest_state_abr],
    'CRS_DEP_TIME': [crs_dep_time],
    'CRS_ARR_TIME': [crs_arr_time],
    'DISTANCE': [distance]
})

# Bouton pour prédire
if st.button('Prédire le Retard'):
    result = predict_delay(input_data)
    if result == 1:
        st.write('Le vol aura probablement du retard.')
    else:
        st.write('Le vol n\'aura probablement pas de retard.')

